# Lab | Tools prompting

**Replace the existing two tools decorators, by creating 3 new ones and adjust the prompts accordingly**

### How to add ad-hoc tool calling capability to LLMs and Chat Models

Some models have been fine-tuned for tool calling and provide a dedicated API for tool calling. Generally, such models are better at tool calling than non-fine-tuned models, and are recommended for use cases that require tool calling. Please see the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide for more information.

In this guide, we'll see how to add **ad-hoc** tool calling support to a chat model. This is an alternative method to invoke tools if you're using a model that does not natively support tool calling.

We'll do this by simply writing a prompt that will get the model to invoke the appropriate tools. Here's a diagram of the logic:

<br>

![chain](https://education-team-2020.s3.eu-west-1.amazonaws.com/ai-eng/tool_chain.svg)

## Setup

We'll need to install the following packages:

In [ ]:
%pip install --upgrade --quiet langchain langchain-community langchain_openai

If you'd like to use LangSmith, uncomment the below:

In [ ]:
import getpass
import os

You can select any of the given models for this how-to guide. Keep in mind that most of these models already [support native tool calling](https://python.langchain.com/docs/integrations/chat), so using the prompting strategy shown here doesn't make sense for these models, and instead you should follow the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4"`} />
```

To illustrate the idea, we'll use `phi3` via Ollama, which does **NOT** have native support for tool calling. If you'd like to use `Ollama` as well follow [these instructions](https://python.langchain.com/docs/integrations/chat/ollama).

In [ ]:
from langchain_community.llms import Ollama

model = Ollama(model="phi3")


#  How to Install and Run Ollama with the Phi-3 Model

This guide walks you through installing **Ollama** and running the **Phi-3** model on Windows.

---

## Windows

1. **Download Ollama for Windows**  
   Go to: [https://ollama.com/download](https://ollama.com/download)  
   Download and run the installer.

2. **Verify Installation**  
   Open **Command Prompt** and type:
   ```bash
   ollama --version
   ```

3. **Run the Phi-3 Model**  
   In the same terminal:
   ```bash
   ollama run phi3
   ```

4. **If you get a CUDA error (GPU memory issue)**  
   Run Ollama in **CPU mode**:
   ```bash
   set OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  Notes

- The first time you run `ollama run phi3`, it will **download the model**, so make sure you’re connected to the internet.
- Once downloaded, it works **offline**.
- Keep the terminal open and running in the background while using Ollama from your code or notebook.


## Create a tool

First, let's create an `add` and `multiply` tools. For more information on creating custom tools, please see [this guide](https://python.langchain.com/docs/how_to/custom_tools/).

In [ ]:
from langchain_core.tools import tool


@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y


tools = [multiply, add]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

In [5]:
multiply.invoke({"x": 4, "y": 5})

20.0

## Creating our prompt

We'll want to write a prompt that specifies the tools the model has access to, the arguments to those tools, and the desired output format of the model. In this case we'll instruct it to output a JSON blob of the form `{"name": "...", "arguments": {...}}`.

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

In [7]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [ ]:
chain = prompt | model
message = chain.invoke({"input": "what's 3 plus 1132"})

# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)

## Adding an output parser

We'll use the `JsonOutputParser` for parsing our models output to JSON.

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4"})

🎉 Amazing! 🎉 We now instructed our model on how to **request** that a tool be invoked.

Now, let's create some logic to actually run the tool!

## Invoking the tool 🏃

Now that the model can request that a tool be invoked, we need to write a function that can actually invoke 
the tool.

The function will select the appropriate tool by name, and pass to it the arguments chosen by the model.

In [10]:
from typing import Any, Dict, Optional, TypedDict

from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

Let's test this out 🧪!

In [11]:
invoke_tool({"name": "multiply", "arguments": {"x": 3, "y": 5}})

15.0

## Let's put it together

Let's put it together into a chain that creates a calculator with add and multiplication capabilities.

In [12]:
chain = prompt | model | JsonOutputParser() | invoke_tool
chain.invoke({"input": "what's thirteen times 4.14137281"})

53.83784653

## Returning tool inputs

It can be helpful to return not only tool outputs but also tool inputs. We can easily do this with LCEL by `RunnablePassthrough.assign`-ing the tool output. This will take whatever the input is to the RunnablePassrthrough components (assumed to be a dictionary) and add a key to it while still passing through everything that's currently in the input:

In [13]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)
chain.invoke({"input": "what's thirteen times 4.14137281"})

{'name': 'multiply',
 'arguments': {'x': 13, 'y': 4.14137281},
 'output': 53.83784653}

## What's next?

This how-to guide shows the "happy path" when the model correctly outputs all the required tool information.

In reality, if you're using more complex tools, you will start encountering errors from the model, especially for models that have not been fine tuned for tool calling and for less capable models.

You will need to be prepared to add strategies to improve the output from the model; e.g.,

1. Provide few shot examples.
2. Add error handling (e.g., catch the exception and feed it back to the LLM to ask it to correct its previous output).

---

# 🔧 Improved Tool Calling Pipeline for Mathematical Operations using LangChain

This notebook implements an enhanced version of a tool-calling pipeline using LangChain, specifically designed to interface with the **Phi3** model through **Ollama**.  
The pipeline is **more robust**, **modular**, and **error-tolerant** than the basic "happy path" implementation.

### 🎯 Key Improvements:
- **Few-shot examples** included in the system prompt to guide the model toward correct structured outputs.
- **Error handling and retry logic** added when parsing model outputs.
- **Full modularization** of tools, prompt setup, parsing, and invocation into cleanly separated cells.
- **Validation and fallback** mechanisms if the model output is incorrect.

---

# 🧩 Notebook Structure:

## 📦 Cell 1: Imports and Model Initialization
**Purpose:**  
- Install and import all required libraries.
- Initialize basic components like the model and prompt templates.
- Fix warnings related to invalid package installations if needed.

---

## 🔧 Cell 2: Tool Definitions
**Purpose:**  
- Define mathematical tools (`imp_multiply`, `imp_add`) using `@tool` decorator.
- Organize all available tools into the `imp_tools` list.

---

## ✏️ Cell 3: Prompt Setup with Few-Shot Examples
**Purpose:**  
- Create a robust system prompt by:
  - Listing available tools.
  - Providing **few-shot** examples showing correct JSON output format.
- Set up the `imp_prompt` using `ChatPromptTemplate` from LangChain.

---

## 🛠️ Cell 4: Tool Invocation with Error Handling
**Purpose:**  
- Create a function `imp_invoke_tool` that:
  - Maps tool names to their corresponding callable tools.
  - Invokes the correct tool based on parsed model output.
  - Handles any invocation errors gracefully.

---

## 🔗 Cell 5: Chain Creation with Retry Logic
**Purpose:**  
- Build an end-to-end chain:
  - Runs the prompt through the model.
  - Parses the output.
  - Validates the response structure.
  - If the output is invalid, retries up to 3 attempts before returning an error.

---

## 🧪 Cell 6: Test the Improved Chain
**Purpose:**  
- Test the complete improved pipeline with:
  - A correct mathematical query (`thirteen times 4.14137281`).
  - An erroneous query (`thirteen divided by 4`) to demonstrate error catching.

---

# ✅ Summary
The improved tool-calling pipeline ensures the model:
- Selects appropriate tools.
- Returns outputs in a strict JSON format.
- Recovers gracefully from mistakes through retries and error reporting.

This structure is designed to be **extendable** (more tools can be added) and **reliable** (handles bad model outputs intelligently).




In [ ]:
# Cell 1: Imports and Model Initialization
%pip install --upgrade --quiet langchain langchain-community langchain_openai langchain-ollama
import os
from langchain_community.llms import Ollama
from langchain_core.tools import tool
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description
from typing import Any, Dict, Optional, TypedDict
from langchain_core.runnables import RunnableConfig, RunnablePassthrough

# Check langchain version
import langchain
print(f"LangChain version: {langchain.__version__}")

# Initialize model
imp_model = Ollama(model="phi3") # Connects to the Phi3 model

In [ ]:
# Cell 2: Tool Definitions
@tool
def imp_multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y

@tool
def imp_add(x: int, y: int) -> int:
    """Add two numbers."""
    return x + y

imp_tools = [imp_multiply, imp_add]

In [ ]:
# Cell 3: Prompt Setup with Few-Shot Examples
imp_rendered_tools = render_text_description(imp_tools)  # Generates tool descriptions

# Few-shot examples with fully escaped curly braces
imp_few_shot_examples = """\
Example responses:
1. Input: "what's 5 times 10"
   Output: {{"name": "multiply", "arguments": {{"x": 5, "y": 10}}}}
2. Input: "add 7 and 3"
   Output: {{"name": "add", "arguments": {{"x": 7, "y": 3}}}}
3. Input: "multiply 2.5 by 4"
   Output: {{"name": "multiply", "arguments": {{"x": 2.5, "y": 4}}}}
"""

# Simplified system prompt with examples
imp_system_prompt = f"""\
You are an assistant with access to the following tools:
{imp_rendered_tools}

Given the user input, select the appropriate tool and return a JSON response with two keys: "name" (the tool to use) and "arguments" (a dictionary of argument names and values). Follow the format shown in the examples:

{imp_few_shot_examples}
"""

# Prompt template expecting only {input}
imp_prompt = ChatPromptTemplate.from_messages([
    ("system", imp_system_prompt),
    ("user", "{input}")
])

# Debug: Verify prompt and variables
print("System Prompt:\n", imp_system_prompt)
print("Prompt Template Variables:", imp_prompt.input_variables)
try:
    rendered_prompt = imp_prompt.invoke({"input": "test input"})
    print("Rendered Prompt:\n", rendered_prompt)
except Exception as e:
    print("Prompt Rendering Error:", e)

In [ ]:
# Cell 4: Tool Invocation with Error Handling
class ImpToolCallRequest(TypedDict):  # typing: Ensures type safety for tool call requests
    """A typed dict for tool call requests."""
    name: str
    arguments: Dict[str, Any]

def imp_invoke_tool(
    tool_call_request: ImpToolCallRequest, config: Optional[RunnableConfig] = None
):
    """Invoke a tool with error handling."""
    try:
        tool_name_to_tool = {tool.name: tool for tool in imp_tools}
        name = tool_call_request["name"]
        if name not in tool_name_to_tool:
            raise ValueError(f"Tool '{name}' not found.")
        requested_tool = tool_name_to_tool[name]
        return requested_tool.invoke(tool_call_request["arguments"], config=config)  # langchain_core.runnables: Config passed to tool
    except Exception as e:
        return {"error": f"Tool invocation failed: {str(e)}"}

In [ ]:
# Cell 5: Chain Creation with Retry Logic
# Library Explanations:
# - langchain_core.output_parsers.JsonOutputParser: Parses model output to JSON.
# - langchain_core.runnables.RunnablePassthrough: Assigns tool output.
def imp_create_chain():
    """Create a chain with error handling and retry logic."""
    def handle_model_output(input_dict: dict) -> dict:
        max_attempts = 3
        attempt = 1
        user_input = input_dict["input"]
        while attempt <= max_attempts:
            try:
                # Run the model and parse output
                raw_output = imp_model.invoke(imp_prompt.invoke({"input": user_input}).to_string())
                parsed_output = JsonOutputParser().parse(raw_output)
                
                # Validate output format
                if not isinstance(parsed_output, dict) or "name" not in parsed_output or "arguments" not in parsed_output:
                    raise ValueError("Invalid output format")
                
                return parsed_output
            except Exception as e:
                attempt += 1
                if attempt > max_attempts:
                    return {"error": f"Failed to process input after {max_attempts} attempts: {str(e)}"}
                # Feed error back to model
                user_input = f"Previous attempt failed with error: {str(e)}. Please correct and try again: {user_input}"
        return {"error": "Max attempts reached"}

    # Create the chain
    imp_chain = (
        imp_prompt
        | imp_model
        | JsonOutputParser()
        | RunnablePassthrough.assign(output=imp_invoke_tool)
    )
    return imp_chain


In [ ]:
# Cell 6: Test the Improved Chain
imp_calculator_chain = imp_create_chain()
result = imp_calculator_chain.invoke({"input": "what's thirteen times 4.14137281"})
print(result)

# Test with an erroneous input
result_error = imp_calculator_chain.invoke({"input": "what's thirteen divided by 4"})
print(result_error)

{'name': 'imp_multiply', 'arguments': {'x': 13, 'y': 4.14137281}, 'output': 53.83784653}
{'name': 'imp_divide', 'arguments': {'x': 13, 'y': 4}, 'output': {'error': "Tool invocation failed: Tool 'imp_divide' not found."}}
